In [9]:
import os
import yaml
import json
import pandas as pd

In [12]:
import wombat.bsub as bsub

In [13]:
root = '/diskmnt/Projects/Users/estorrs/pecgs-pipeline'

load sample data

In [18]:
variant_filepath = '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/druggability/test.maf'
    
annotate_trials_keyword = '' # empty until you get files from jay
tumor_sample_name = 'C3L-00677.T'
normal_sample_name = 'C3L-00677.N'
sample = 'C3L-00677'

make test input yaml

In [19]:
template = yaml.safe_load(open(f'{root}/submodules/pecgs-druggability/cwl/template.druggability.yaml'))
template


{'annotate_trials_keyword': 'a_string',
 'normal_sample_name': 'a_string',
 'tumor_sample_name': 'a_string',
 'variant_file_type': 'a_string',
 'variant_filepath': {'class': 'File', 'path': 'a/file/path'}}

In [20]:
template['variant_filepath']['path'] = variant_filepath

template['tumor_sample_name'] = tumor_sample_name
template['normal_sample_name'] = normal_sample_name
template['annotate_trials_keyword'] = annotate_trials_keyword
template.pop('variant_file_type')


'a_string'

In [21]:
template

{'annotate_trials_keyword': '',
 'normal_sample_name': 'C3L-00677.N',
 'tumor_sample_name': 'C3L-00677.T',
 'variant_filepath': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/druggability/test.maf'}}

In [22]:
yaml.safe_dump(template, open(f'../tests/data/druggability/{sample}_inputs.yaml', 'w'))

make cromwell compute1 config

In [23]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [24]:
workflow_root = f'/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/druggability/{sample}'
output_fp = f'../tests/data/druggability/{sample}.cromwell-config-db.compute1.dat'
args['queue'] = 'general'
bsub.save_compute1_cromwell_template(workflow_root, output_fp, queue=args['queue'])

make execution script

In [25]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/druggability'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, f'{sample}.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-druggability', 'cwl', 'druggability.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, f'{sample}_inputs.yaml')
log_fp = os.path.join(TEST_DIR_ROOT, f'{sample}.log')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/scratch1/fs1/dinglab/estorrs', '/home/estorrs']

In [26]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes,
                                     workflow_root=workflow_root, interactive=False,
                                     log_fp=log_fp)
filepath = os.path.join('../tests/data/druggability/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [27]:
run_commands

['mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/druggability/C3L-00677/cromwell-workdir/logs',
 'source /opt/ibm/lsfsuite/lsf/conf/lsf.conf',
 'export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/druggability:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/druggability /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/druggability/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/druggability/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs /home/estorrs:/home/estorrs"',
 'bgadd -L 10 /estorrs/test_cromwell',
 'export PATH="/opt/java/openjdk/bin:$PATH"',
 "bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q general -G compute-dingla